In [34]:
import logging
import apache_beam as beam

from apache_beam.options.pipeline_options import PipelineOptions


In [35]:
# logging
root = logging.getLogger()
root.setLevel(logging.INFO)


In [36]:
# create a pipeline, read data from csv
pipeline_options = beam.Pipeline(options=PipelineOptions())

# PCollection format is = id, customer-type, product-type, price
transactions = (pipeline_options
                | "Read Transaction CSV"
                >> beam.io.ReadFromText("gs://spark-dataset-1/datasets/sales/sales_transactions.csv")
                )

(transactions
     | "Raw transactions"
     >> beam.CombineGlobally(beam.combiners.CountCombineFn())
     | "Raw transactions count"
     >> beam.ParDo(lambda c: logging.info("Total lines %s", str(c)))
)


INFO:root:Missing pipeline option (runner). Executing pipeline using the default runner: DirectRunner.
INFO:oauth2client.client:Timeout attempting to reach GCE metadata service.
Connecting anonymously.
INFO:oauth2client.client:Timeout attempting to reach GCE metadata service.
Connecting anonymously.


<PCollection[Raw transactions count.None] at 0x119881c10>

In [37]:
# extract product-type and price
class ExtractProductTypePrice(beam.DoFn):

    def process(self, element, *args, **kwargs):
        line = element.split(',')
        return [(line[2], float(line[3]))]


# PCollection format now = product-type, price
productType_price = (transactions
                     | "Extracting product type and price"
                     >> beam.ParDo(ExtractProductTypePrice())          
                     )

(productType_price
 | "Product Type and Price"
 >> beam.CombineGlobally(beam.combiners.CountCombineFn())
 | "Product Type and Price count"
 >> beam.ParDo(lambda c: logging.info("Total lines %s", str(c)))  
)


<PCollection[Product Type and Price count.None] at 0x119898ed0>

In [38]:
# run
result = pipeline_options.run()

result.wait_until_finish()

INFO:root:==================== <function annotate_downstream_side_inputs at 0x11712a230> ====================
INFO:root:==================== <function fix_side_input_pcoll_coders at 0x11712a2a8> ====================
INFO:root:==================== <function lift_combiners at 0x11712a9b0> ====================
INFO:root:==================== <function expand_gbk at 0x11712a410> ====================
INFO:root:==================== <function sink_flattens at 0x11712a320> ====================
INFO:root:==================== <function greedily_fuse at 0x11712a758> ====================
INFO:root:==================== <function impulse_to_input at 0x11712a668> ====================
INFO:root:==================== <function inject_timer_pcollections at 0x1173c0668> ====================
INFO:root:==================== <function sort_stages at 0x1173c06e0> ====================
INFO:root:Running ((((ref_AppliedPTransform_Read Transaction CSV/Read_3)+((ref_AppliedPTransform_Raw transactions/KeyWithVoid_5)+

'DONE'